In [118]:
import os    
from dotenv  import load_dotenv
from pathlib import Path
import requests
import pandas as pd
pd.options.display.max_colwidth = 600  # Set Full Display

import datetime
from dateutil import relativedelta
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

import ast
# import json

In [2]:
suburb="Glen_Waverley"
suburb_search = "glen-waverley-vic-3150"

# History Analysis:

### Initialization: Loading saved data

In [10]:
# use r -- to avoid unicode error !
df_history_path=Path(f'Address_Code_History_Combined_{suburb}.xlsx')
df_history = pd.read_excel(df_history_path, parse_dates=True)

In [11]:
history_df = df_history[["address_check","recent_history"]]
print(history_df.info())
history_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 2 columns):
address_check     446 non-null object
recent_history    379 non-null object
dtypes: object(2)
memory usage: 7.2+ KB
None


,address_check,recent_history
28,17-latham-ct-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2009-11-14', 'price': 900000, 'agency': 'Harcourts Glen Waverley'}"
207,3-chesterville-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2002-10-23', 'price': 0}"
196,76-winmalee-dr-glen-waverley-vic-3150,NaN
422,42a-hampshire-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1981-05-01', 'price': 14950}"
204,9-olympic-ct-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2018-05-12', 'price': 3380000, 'agency': 'Ray White Glen Waverley'}"


排除掉 Null:

In [32]:
history_df2=history_df[history_df["recent_history"].notnull()]
history_df2

,address_check,recent_history
0,unit-1-22-melaleuca-dr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2005-09-11', 'price': 391000, 'agency': 'Barry Plant Doherty Real Estate Glen Waverley'}"
1,unit-1-34-montclair-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2015-03-20', 'price': 0}"
2,10-wedge-ct-glen-waverley-vic-3150,"{'event': 'Rent', 'date': '2017-07-12', 'price': 500, 'agency': 'Ray White Real Estate Glen Waverley'}"
3,unit-4-47-nottingham-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2010-02-25', 'price': 620000, 'agency': 'Hocking Stuart Glen Waverley'}"
5,7-cooper-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2004-09-28', 'price': 312500}"
...,...,...
445,17-boronia-dr-glen-waverley-vic-3150,"{'event': 'Rent', 'date': '2017-03-14', 'price': 480, 'agency': 'For Rent By Owner'}"
446,unit-2-27-29-botanic-dr-glen-waverley-vic-3150,"{'event': 'Rent', 'date': '2015-06-09', 'price': 855, 'agency': 'For Rent By Agency'}"
447,20-margate-cres-glen-waverley-vic-3150,"{'event': 'Rent', 'date': '2018-02-13', 'price': 490, 'agency': 'Jellis Craig - Mount Waverley'}"
448,unit-1-1-martin-pl-glen-waverley-vic-3150,"{'event': 'Rent', 'date': '2015-04-20', 'price': 680, 'agency': 'Harcourts Glen Waverley'}"


篩選 Sold:

In [98]:
history_df3 = history_df2[history_df2["recent_history"].str.contains("Sold")]
history_df3 = history_df3.reset_index(drop=True)
history_df3

,address_check,recent_history
0,unit-1-22-melaleuca-dr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2005-09-11', 'price': 391000, 'agency': 'Barry Plant Doherty Real Estate Glen Waverley'}"
1,unit-1-34-montclair-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2015-03-20', 'price': 0}"
2,unit-4-47-nottingham-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2010-02-25', 'price': 620000, 'agency': 'Hocking Stuart Glen Waverley'}"
3,7-cooper-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2004-09-28', 'price': 312500}"
4,33-willow-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1989-10-01', 'price': 110000}"
...,...,...
288,unit-3-802-highbury-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2021-12-08', 'price': 903000, 'agency': 'Qunita Real Estate - VERMONT SOUTH'}"
289,16-belmont-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2016-02-28', 'price': 800000}"
290,8-swift-dr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1988-06-01', 'price': 138000}"
291,2-golden-gr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2017-03-13', 'price': 1640000, 'agency': 'Harcourts Glen Waverley'}"


提取日期：String

In [99]:
sold_date=[]
for i, row in history_df3.iterrows(): # print(i) # row number
    sold_date.append(ast.literal_eval(row["recent_history"])['date'])

sold_date_df=pd.DataFrame(sold_date)   
sold_date_df.columns = ["Date"]
sold_date_df = sold_date_df.reset_index(drop=True)
sold_date_df


,Date
0,2005-09-11
1,2015-03-20
2,2010-02-25
3,2004-09-28
4,1989-10-01
...,...
288,2021-12-08
289,2016-02-28
290,1988-06-01
291,2017-03-13


重組：

非常注意，兩個 df 必須 reset index -- 不然會 miss match !!

In [100]:
history_df4 = pd.concat([history_df3,sold_date_df],axis=1)
history_df4

,address_check,recent_history,Date
0,unit-1-22-melaleuca-dr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2005-09-11', 'price': 391000, 'agency': 'Barry Plant Doherty Real Estate Glen Waverley'}",2005-09-11
1,unit-1-34-montclair-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2015-03-20', 'price': 0}",2015-03-20
2,unit-4-47-nottingham-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2010-02-25', 'price': 620000, 'agency': 'Hocking Stuart Glen Waverley'}",2010-02-25
3,7-cooper-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2004-09-28', 'price': 312500}",2004-09-28
4,33-willow-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1989-10-01', 'price': 110000}",1989-10-01
...,...,...,...
288,unit-3-802-highbury-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2021-12-08', 'price': 903000, 'agency': 'Qunita Real Estate - VERMONT SOUTH'}",2021-12-08
289,16-belmont-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2016-02-28', 'price': 800000}",2016-02-28
290,8-swift-dr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1988-06-01', 'price': 138000}",1988-06-01
291,2-golden-gr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2017-03-13', 'price': 1640000, 'agency': 'Harcourts Glen Waverley'}",2017-03-13


轉換 Datatime:

In [102]:
history_df4["Date"] = pd.to_datetime(history_df4["Date"])
history_df4

,address_check,recent_history,Date
0,unit-1-22-melaleuca-dr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2005-09-11', 'price': 391000, 'agency': 'Barry Plant Doherty Real Estate Glen Waverley'}",2005-09-11
1,unit-1-34-montclair-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2015-03-20', 'price': 0}",2015-03-20
2,unit-4-47-nottingham-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2010-02-25', 'price': 620000, 'agency': 'Hocking Stuart Glen Waverley'}",2010-02-25
3,7-cooper-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2004-09-28', 'price': 312500}",2004-09-28
4,33-willow-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1989-10-01', 'price': 110000}",1989-10-01
...,...,...,...
288,unit-3-802-highbury-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2021-12-08', 'price': 903000, 'agency': 'Qunita Real Estate - VERMONT SOUTH'}",2021-12-08
289,16-belmont-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2016-02-28', 'price': 800000}",2016-02-28
290,8-swift-dr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1988-06-01', 'price': 138000}",1988-06-01
291,2-golden-gr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2017-03-13', 'price': 1640000, 'agency': 'Harcourts Glen Waverley'}",2017-03-13


In [125]:
# Check Date type:
history_df4.iloc[5]["Date"]

Timestamp('1987-01-01 00:00:00')

篩選，時間到現在間隔 5年以上的：

In [106]:
now_date = datetime.datetime.now()
now_date

datetime.datetime(2022, 4, 1, 1, 5, 29, 199814)

In [124]:
# 5年前：
fiveyrsago = now_date - relativedelta(years=5)
benckmark=datetime.strftime(fiveyrsago,"%Y-%m-%d")
# benckmark=datetime.strptime(benckmark,"%Y-%m-%d")
benckmark

'2017-04-01'

選擇 5 年以上的數據：

In [132]:
history_df5 = history_df4[history_df4["Date"]<benckmark]
history_df5 = history_df5.sort_values(by=["Date"],ascending = False)
history_df5

,address_check,recent_history,Date
151,18-ingleside-cres-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2017-03-25', 'price': 2388000, 'agency': 'Jrw Property International - Glen Waverley'}",2017-03-25
27,5-aurora-ct-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2017-03-24', 'price': 0}",2017-03-24
109,unit-3-14-myers-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2017-03-14', 'price': 815000, 'agency': 'Harcourts Box Hill Tsl'}",2017-03-14
291,2-golden-gr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2017-03-13', 'price': 1640000, 'agency': 'Harcourts Glen Waverley'}",2017-03-13
163,32-kinnoull-gr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2016-12-03', 'price': 1860000, 'agency': 'Barry Plant Glen Waverley'}",2016-12-03
...,...,...,...
158,9-florence-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1979-02-16', 'price': 33000}",1979-02-16
270,10-maylands-cres-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1978-03-22', 'price': 43000}",1978-03-22
67,6-tamarisk-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1975-08-13', 'price': 34500}",1975-08-13
146,42-mount-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1975-05-22', 'price': 28250}",1975-05-22


In [133]:
# create excel writer object
writer=pd.ExcelWriter(f'History_Analysis_{suburb}.xlsx')
# write dataframe to excel
history_df5.to_excel(writer, index=False)
# save the excel
writer.save()
print('DataFrame is written successfully to Excel File.')

DataFrame is written successfully to Excel File.


# Suburb For Sale Properties: Days on the market

In [12]:
# use r -- to avoid unicode error !
filesaved_forsale_path=Path(rf'C:\Users\LEON\test\domain\Suburb_for_Sale_{suburb}.xlsx')
filesaved_forsale_df = pd.read_excel(filesaved_forsale_path, parse_dates=True)


In [13]:
for_sale_df = filesaved_forsale_df[["address","suburb_name","bed","bath","car","list_date","list_price","days_on_market"]]
print(for_sale_df.info())
for_sale_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 8 columns):
address           181 non-null object
suburb_name       181 non-null object
bed               181 non-null int64
bath              181 non-null int64
car               181 non-null int64
list_date         181 non-null object
list_price        72 non-null object
days_on_market    181 non-null int64
dtypes: int64(4), object(4)
memory usage: 11.4+ KB
None


,address,suburb_name,bed,bath,car,list_date,list_price,days_on_market
170,813/39 Kingsway,Glen Waverley,1,1,1,2021-09-12,"$345,000 to $350,000",197
68,1106/52-54 O'Sullivan Road,Glen Waverley,2,1,1,2022-02-25,"$570,000-$630,000 Private Sale",31
107,27 Wilson Road,Glen Waverley,5,5,2,2021-12-08,NaN,110
88,2/11 Pindari Street,Glen Waverley,4,4,2,2022-02-07,NaN,49
4,2/62 Cypress Avenue,Glen Waverley,3,2,2,2022-03-25,NaN,3


In [25]:
for_sale_df2=for_sale_df[for_sale_df["days_on_market"]>=50]
for_sale_df2

,address,suburb_name,bed,bath,car,list_date,list_price,days_on_market
89,1102/39-55 Kingsway,Glen Waverley,1,1,1,2022-02-03,NaN,53
90,306/286-290 Blackburn Road,Glen Waverley,2,1,1,2022-02-02,"$500,000 - $550,000",54
91,7 Hinkler Road,Glen Waverley,3,1,2,2022-02-02,NaN,54
92,37B Charlotte Street,Glen Waverley,4,3,2,2022-02-01,"$1,875,000",55
93,572 Highbury Road,Glen Waverley,5,5,2,2022-01-28,NaN,59
...,...,...,...,...,...,...,...,...
176,Type 130/23 O'Sullivan Road,Glen Waverley,3,2,2,2018-06-22,"$1,260,000",1375
177,Type 123/23 O'Sullivan Road,Glen Waverley,3,2,2,2018-06-22,"$1,180,000",1375
178,Type 327/23 O'Sullivan Road,Glen Waverley,3,2,2,2018-06-22,"$1,180,000",1375
179,Type 116.1/23 O'Sullivan Road,Glen Waverley,2,2,1,2018-06-22,"$698,000",1375


In [26]:
# create excel writer object
writer=pd.ExcelWriter(f'For_Sale_Analysis_{suburb}.xlsx')
# write dataframe to excel
for_sale_df2.to_excel(writer, index=False)
# save the excel
writer.save()
print('DataFrame is written successfully to Excel File.')

DataFrame is written successfully to Excel File.


# Suburb Auction Results: 半年範圍，但是沒有 20 Cherry 説明 並不完整 ！！

'Sold',

'Passed In',

'Private Sale',

'Withdrawn Prior To Auction',

In [16]:
# use r -- to avoid unicode error !
filesaved_auction_path=Path(rf'C:\Users\LEON\test\domain\Auction_Result_{suburb}.xlsx')
filesaved_auction_df = pd.read_excel(filesaved_auction_path, parse_dates=True)
print(filesaved_auction_df.info())
filesaved_auction_df.tail(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 13 columns):
address        155 non-null object
agency         155 non-null object
bed            155 non-null int64
id             155 non-null int64
is_sold        155 non-null bool
postcode       155 non-null int64
price          155 non-null int64
pu_id          101 non-null float64
result         155 non-null object
sold_date      155 non-null object
state          155 non-null object
suburb_name    155 non-null object
type           155 non-null object
dtypes: bool(1), float64(1), int64(4), object(7)
memory usage: 14.8+ KB
None


,address,agency,bed,id,is_sold,postcode,price,pu_id,result,sold_date,state,suburb_name,type
153,561 Waverley Rd,Biggin & Scott Glen Waverley,4,421366860,True,3150,1402000,NaN,Private Sale,2021-10-02,VIC,Glen Waverley,house
154,1/577 Waverley Rd,Biggin & Scott Glen Waverley,4,421076860,True,3150,1420000,NaN,Private Sale,2021-10-02,VIC,Glen Waverley,townhouse


In [18]:
auction_result_df = filesaved_auction_df[["address","type","bed","agency","is_sold","price","result"]]
print(auction_result_df.info())
auction_result_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 7 columns):
address    155 non-null object
type       155 non-null object
bed        155 non-null int64
agency     155 non-null object
is_sold    155 non-null bool
price      155 non-null int64
result     155 non-null object
dtypes: bool(1), int64(2), object(4)
memory usage: 7.5+ KB
None


,address,type,bed,agency,is_sold,price,result
152,22 Tamarisk Av,house,4,First National Quinta,True,1570000,Private Sale
16,12A Oleander St,townhouse,4,Barry Plant Monash,True,1396000,Sold
114,67 Herriotts Bvd,house,5,Barry Plant Monash,True,512000,Sold
27,4 Stanton Ct,house,5,Biggin & Scott Glen Waverley,True,1780000,Sold
131,31 Guinevere Pde,house,3,Ray White Blackburn,True,1270000,Sold Prior


非常注意：

一些房子先 pass in 但是後來賣掉！

In [60]:
# DataFrame.duplicated(subset=None, keep='first')

In [63]:
auction_result_df3 = auction_result_df[auction_result_df["address"].duplicated(keep=False)] #顯示全部重複的
auction_result_df3

,address,type,bed,agency,is_sold,price,result
4,14 Holmes Wy,house,4,Biggin & Scott Glen Waverley,True,1830000,Private Sale
37,14 Holmes Wy,house,4,Biggin & Scott Glen Waverley,False,1800000,Passed In
45,4/2 Pimm Ct,unit,2,Biggin & Scott Glen Waverley,True,745000,Private Sale
49,4/2 Pimm Ct,unit,2,Biggin & Scott Glen Waverley,True,745000,Private Sale
57,2/671 High Street Rd,townhouse,4,Barry Plant Monash,True,1510000,Private Sale
91,17 Kiah St,house,4,LLC Real Estate,True,2700000,Private Sale
101,17 Kiah St,house,4,LLC Real Estate,True,2700000,Private Sale
108,20 Highvale Rd,house,6,Barry Plant Monash,True,3018888,Private Sale
109,1/9 Kurrajong Av,townhouse,4,First National Avant,True,1250000,Private Sale
115,2/671 High Street Rd,townhouse,4,Barry Plant Monash,False,0,Passed In


非常注意：這個是所有重複地址的，且都賣掉的，即使之前沒賣掉。

In [65]:
auction_result_df4 = auction_result_df3[auction_result_df3["is_sold"]==True]
auction_result_df4

,address,type,bed,agency,is_sold,price,result
4,14 Holmes Wy,house,4,Biggin & Scott Glen Waverley,True,1830000,Private Sale
45,4/2 Pimm Ct,unit,2,Biggin & Scott Glen Waverley,True,745000,Private Sale
49,4/2 Pimm Ct,unit,2,Biggin & Scott Glen Waverley,True,745000,Private Sale
57,2/671 High Street Rd,townhouse,4,Barry Plant Monash,True,1510000,Private Sale
91,17 Kiah St,house,4,LLC Real Estate,True,2700000,Private Sale
101,17 Kiah St,house,4,LLC Real Estate,True,2700000,Private Sale
108,20 Highvale Rd,house,6,Barry Plant Monash,True,3018888,Private Sale
109,1/9 Kurrajong Av,townhouse,4,First National Avant,True,1250000,Private Sale
123,1/9 Kurrajong Av,townhouse,4,First National Avant,True,1250000,Private Sale
130,22 Tamarisk Av,house,4,First National Quinta,True,1570000,Private Sale


In [23]:
auction_result_df2=auction_result_df[auction_result_df["is_sold"]!=True]
auction_result_df2

,address,type,bed,agency,is_sold,price,result
1,25 Cooper Av,house,5,Barry Plant Monash,False,0,Passed In
2,36 Danien St,house,3,Barry Plant Monash,False,0,Passed In
3,2/10 Kurrajong Av,townhouse,3,Harcourts Judd White,False,0,Passed In
26,2/11 Pindari St,townhouse,4,Barry Plant Monash,False,0,Passed In
31,572 Highbury Rd,house,5,Fletchers Waverley,False,0,Passed In
37,14 Holmes Wy,house,4,Biggin & Scott Glen Waverley,False,1800000,Passed In
41,27 Gauntlet Av,house,4,Ray White Balwyn,False,0,Passed In
50,6 Alimar Rd,house,4,Biggin & Scott Glen Waverley,False,0,Withdrawn Prior To Auction
60,3/11 Aurisch Av,townhouse,4,Barry Plant Monash,False,0,Passed In
70,12 Caithness Cr,house,6,McGrath Box Hill,False,0,Passed In


非常注意：如果沒賣掉table中含有，後來被賣掉的，那就篩除！

非常注意 ~ 是 NOT ！！

In [70]:
# ~auction_result_df2.address.isin(auction_result_df4.address)

In [71]:
auction_result_df5=auction_result_df2[~auction_result_df2.address.isin(auction_result_df4.address)]
auction_result_df5

,address,type,bed,agency,is_sold,price,result
1,25 Cooper Av,house,5,Barry Plant Monash,False,0,Passed In
2,36 Danien St,house,3,Barry Plant Monash,False,0,Passed In
3,2/10 Kurrajong Av,townhouse,3,Harcourts Judd White,False,0,Passed In
26,2/11 Pindari St,townhouse,4,Barry Plant Monash,False,0,Passed In
31,572 Highbury Rd,house,5,Fletchers Waverley,False,0,Passed In
41,27 Gauntlet Av,house,4,Ray White Balwyn,False,0,Passed In
50,6 Alimar Rd,house,4,Biggin & Scott Glen Waverley,False,0,Withdrawn Prior To Auction
60,3/11 Aurisch Av,townhouse,4,Barry Plant Monash,False,0,Passed In
70,12 Caithness Cr,house,6,McGrath Box Hill,False,0,Passed In
72,2/2 Peveril St,townhouse,4,McGrath Waverley,False,0,Withdrawn Prior To Auction


In [72]:
# create excel writer object
writer=pd.ExcelWriter(f'Auction_Analysis_{suburb}.xlsx')
# write dataframe to excel
auction_result_df5.to_excel(writer, index=False)
# save the excel
writer.save()
print('DataFrame is written successfully to Excel File.')

DataFrame is written successfully to Excel File.
